In [ ]:
%pip install torch
%pip install torchdata
%pip install numpy
%pip intsall transformers
%pip install datasets
%pip install rouge_score
%pip install evaluate
%pip install loralib
%pip install peft

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import pandas as pd
import numpy as np
import torch
import time
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import evaluate

In [ ]:
model_name = 'google/pegasus-xsum'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype = torch.bfloat16)


# Tokenization

tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [ ]:
def get_num_trainable_params(model):
  total_params = 0
  total_trainable_params = 0

  for _ , params in model.named_parameters():
    total_params += params.numel()

    if params.requires_grad:
      total_trainable_params += params.numel()

  return f"Trainable Param = {total_trainable_params}\nTotal Params = {total_params}\n% of trainable params = {100*(total_trainable_params/total_params)}"

print(get_num_trainable_params(original_model))

Trainable Param = 568699904
Total Params = 569748480
% of trainable params = 99.81595808733005


In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

dataset

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [ ]:
def tokenize_function(example):
  start_prompt = "Symmarize following conversations:\n"
  end_prompt = "\nSumamry:\n"
  prompt = [start_prompt+dialogue+end_prompt for dialogue in example['dialogue']]
  example['input_ids'] = tokenizer(prompt, padding= 'max_length', truncation = True, return_tensors='pt').input_ids
  example['labels'] = tokenizer(example['summary'], padding= 'max_length', truncation = True, return_tensors='pt').input_ids

  return example


tokenized_dataset = dataset.map(tokenize_function, batched = True)
tokenized_dataset


Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'labels'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'labels'],
        num_rows: 1500
    })
})

In [ ]:
tokenized_dataset = tokenized_dataset.remove_columns(['id', 'dialogue', 'summary', 'topic'])
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1500
    })
})

In [ ]:
print(f"Train data shape = {tokenized_dataset['train'].shape}")
print(f"validation data shape = {tokenized_dataset['validation'].shape}")
print(f"Test data shape = {tokenized_dataset['test'].shape}")

Train data shape = (12460, 2)
validation data shape = (500, 2)
Test data shape = (1500, 2)


In [ ]:
tokenized_dataset = tokenized_dataset.filter(lambda example, index: index % 10 == 0, with_indices = True)

print(f"Train data shape = {tokenized_dataset['train'].shape}")
print(f"validation data shape = {tokenized_dataset['validation'].shape}")
print(f"Test data shape = {tokenized_dataset['test'].shape}")
print(tokenized_dataset)

Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1500 [00:00<?, ? examples/s]

Train data shape = (1246, 2)
validation data shape = (50, 2)
Test data shape = (150, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1246
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 50
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 150
    })
})


In [ ]:
print(original_model)

PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(512, 1024)
      (layers): ModuleList(
        (0-15): 16 x PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_nor

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, #Rank
    lora_alpha=32,
    target_modules=['q_proj',
                    'v_proj',],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type=TaskType.SEQ_2_SEQ_LM   #FLAN-T5
)


In [ ]:
peft_model = get_peft_model(original_model, lora_config)

print(get_num_trainable_params(peft_model))

Trainable Param = 6291456
Total Params = 576039936
% of trainable params = 1.092190941428061


In [ ]:
output_dir = f'./pegasus-peft-lora-dialogsum'

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    auto_find_batch_size = True,
    learning_rate = 1e-3,
    num_train_epochs = 2,
    logging_steps = 100,)

peft_trainer = Trainer(model = peft_model,
                      args = peft_training_args,
                      train_dataset = tokenized_dataset['train'])

In [ ]:
# peft_trainer.train(resume_from_checkpoint=True)

peft_trainer.train()
peft_model_path =  "./peft-lora-dialogsum-checkpoints"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (0) is identical to the `bos_token_id` (0), `eos_token_id` (1), or the `sep_token_id` (None), and your input is not padded.


Step,Training Loss
100,8.915300
200,7.332800
300,7.214400
400,7.175000
500,7.194700
600,7.169700


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('./peft-lora-dialogsum-checkpoints/tokenizer_config.json',
 './peft-lora-dialogsum-checkpoints/special_tokens_map.json',
 './peft-lora-dialogsum-checkpoints/spiece.model',
 './peft-lora-dialogsum-checkpoints/added_tokens.json',
 './peft-lora-dialogsum-checkpoints/tokenizer.json')

In [ ]:
device = original_model.device

dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    print(idx)
    prompt = f"""
    Summarize the following conversations :

    {dialogue}

    Summary :
    """
    input_ids = tokenizer(prompt, return_tensors = 'pt').input_ids

    human_baseline_text_output = human_baseline_summaries[idx]

    original_model_output = original_model.generate(input_ids = input_ids.to(device),
                                                generation_config = GenerationConfig(max_new_tokens = 50))
    original_model_text_output = tokenizer.decode(original_model_output[0], skip_special_tokens = True)

    peft_model_output = peft_model.generate(input_ids = input_ids.to(device),
                                                generation_config = GenerationConfig(max_new_tokens = 50))
    peft_model_text_output = tokenizer.decode(peft_model_output[0], skip_special_tokens = True)

    original_model_summaries.append(original_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'peft_model_summaries'])

df

0
1
2
3
4
5
6
7
8
9


,human_baseline_summaries,original_model_summaries,peft_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,#Person1# tells #Person2# to take a break and ...,#Person1# wants to take a dictation for #Perso...
1,In order to prevent employees from wasting tim...,#Person1# wants to take a dictation. #Person1#...,#Person1# asks #Person2# to take a dictation. ...
2,Ms. Dawson takes a dictation for #Person1# abo...,#Person1# tells #Person2# that the company is ...,#Person2# wants to take a dictation for #Perso...
3,#Person2# arrives late because of traffic jam....,#Person2# tells #Person1# #Person1# that #Pers...,#Person1# tells #Person2# #Person1# about the ...
4,#Person2# decides to follow #Person1#'s sugges...,#Person1# tells #Person2# that #Person1# is a ...,#Person1# and #Person2# are on the way to the ...
5,#Person2# complains to #Person1# about the tra...,#Person1# and #Person2# are discussing the tra...,#Person1# tells #Person2# that she's going to ...
6,#Person1# tells Kate that Masha and Hero get d...,#Person2# calls Hero and Masha and Hero tells ...,Kate and Hero are separated. Hero is not happy...
7,#Person1# tells Kate that Masha and Hero are g...,Kate calls Hero and Masha. Masha is not happy....,#Person1# tells #Person2# Hero is getting divo...
8,#Person1# and Kate talk about the divorce betw...,Masha and Hero are getting divorced.' cultivar...,Masha and Hero are getting divorced. Hero's pa...
9,#Person1# and Brian are at the birthday party ...,Brian's birthday is coming up and Brian's goin...,#Person2# is celebrating Brian's birthday. #Pe...


In [ ]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('PEFT Model:')
print(original_model_results)
print('Original Model:')
print(peft_model_results)

PEFT Model:
{'rouge1': 0.1960458975298376, 'rouge2': 0.029463931845733808, 'rougeL': 0.1458909515691546, 'rougeLsum': 0.14658065447269653}
Original Model:
{'rouge1': 0.18098017644738956, 'rouge2': 0.03304755513556203, 'rougeL': 0.15293770000653856, 'rougeLsum': 0.15298550332115213}


In [ ]:
print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(original_model_results.values())) - np.array(list(peft_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100}%')

Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL
rouge1: 1.5065721082448025%
rouge2: 0.3583623289828225%
rougeL: 0.7046748437383954%
rougeLsum: 0.6404848848455602%


In [ ]:
%pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.8 MB/s eta 0:00:00


In [ ]:
import sacrebleu

original_model_references = human_baseline_summaries[0:len(original_model_summaries)]
peft_model_references = human_baseline_summaries[0:len(peft_model_summaries)]

original_model_bleu = sacrebleu.corpus_bleu(original_model_summaries, original_model_references)
peft_model_bleu = sacrebleu.corpus_bleu(peft_model_summaries, peft_model_references)

print('Original Model BLEU:', original_model_bleu.score)
print('PEFT Model BLEU:', peft_model_bleu.score)


Original Model BLEU: 0.09602403633577834
PEFT Model BLEU: 0.09713359476346771


In [ ]:
!zip -r /content/pegasus-peft-lora-dialogsum.zip /content/pegasus-peft-lora-dialogsum
!zip -r /content/peft-lora-dialogsum-checkpoints.zip /content/peft-lora-dialogsum-checkpoints



  adding: content/pegasus-peft-lora-dialogsum/ (stored 0%)
  adding: content/pegasus-peft-lora-dialogsum/checkpoint-500/ (stored 0%)
  adding: content/pegasus-peft-lora-dialogsum/checkpoint-500/trainer_state.json (deflated 65%)
  adding: content/pegasus-peft-lora-dialogsum/checkpoint-500/optimizer.pt (deflated 23%)
  adding: content/pegasus-peft-lora-dialogsum/checkpoint-500/README.md (deflated 66%)
  adding: content/pegasus-peft-lora-dialogsum/checkpoint-500/adapter_config.json (deflated 52%)
  adding: content/pegasus-peft-lora-dialogsum/checkpoint-500/rng_state.pth (deflated 25%)
  adding: content/pegasus-peft-lora-dialogsum/checkpoint-500/adapter_model.safetensors (deflated 22%)
  adding: content/pegasus-peft-lora-dialogsum/checkpoint-500/training_args.bin (deflated 51%)
  adding: content/pegasus-peft-lora-dialogsum/checkpoint-500/scheduler.pt (deflated 55%)
  adding: content/pegasus-peft-lora-dialogsum/runs/ (stored 0%)
  adding: content/pegasus-peft-lora-dialogsum/runs/May29_12-01

In [ ]:
from google.colab import files
files.download('/content/pegasus-peft-lora-dialogsum.zip')
files.download('/content/peft-lora-dialogsum-checkpoints.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>